# Login HuggingFace to Upload Model

In [2]:
# !git config --global credential.helper store

# from huggingface_hub import notebook_login
# notebook_login()

# 1. Training

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
!nvidia-smi

Tue Mar  8 12:49:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   39C    P0    34W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   41C    P0    36W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [5]:
def print_encoded(encoded_inputs):
    for ts in encoded_inputs:
        print(ts, encoded_inputs[ts])

## 1.1 Prepare-Datasets

In [6]:
import transformers
print(transformers.__version__)

4.17.0


In [7]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-mul"

In [8]:
from datasets import load_dataset, load_metric, DatasetDict, Dataset

# raw_datasets = load_dataset("scb_mt_enth_2020", name="then")
metric = load_metric("sacrebleu")

In [9]:
import pandas as pd
def pre_process_from_csv(path, n_row=100000):
    df_5M = pd.read_csv(path, nrows=n_row)
    list_5M = df_5M.to_dict('records')
    list_sub = ['LST_Corpus']*len(list_5M)
    dict_5M = pd.DataFrame({"translation": list_5M, "subdataset": list_sub})
    return dict_5M

In [10]:
raw_datasets = pre_process_from_csv('TEST_dataset_5M.csv', 1000000) # 1Mill to the top

In [11]:
raw_datasets['translation']

0         {'th': 'ขอบคุณ แต่ ฉัน ไม่ มี เวลา พอ', 'en': ...
1         {'th': 'โอ้ , คุณ รู้ , เมื่อ ฉัน ตื่น ฉัน จะ ...
2         {'th': 'หน้าที่ ของ ตำรวจ คือ การ ปกป้อง ประชา...
3         {'th': 'มัน เป็นความ โอหัง ของ เพศ หญิง .', 'e...
4         {'th': 'โปรด คอย ดู อย่า เผลอ', 'en': 'please ...
                                ...                        
999995    {'th': 'คาด ​ ว่า ​ การ ​ ประชุม ​ ทั้ง ​ หมด ...
999996    {'th': 'หรือ ​ กว่า ​ นั้น ​ เล็ก ​ น้อย .', '...
999997    {'th': 'จะ ​ ไม่ ​ มี ​ การ ​ ประชุม ​ ภาค ​ บ...
999998    {'th': 'เนื่อง ​ จาก ​ สถาน ​ ที่ ​ มี ​ จำกัด...
999999    {'th': 'จะ ​ ไม่ ​ มี ​ การ ​ จัด ​ เตรียม ​ ใ...
Name: translation, Length: 1000000, dtype: object

In [12]:
cut_datasets = DatasetDict()
cut_datasets = Dataset.from_pandas(raw_datasets, split="train+validation").train_test_split(0.1)

In [13]:
cut_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 900000
    })
    test: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 100000
    })
})

In [14]:
import datasets, random
import pandas as pd
from IPython.display import display, HTML

def show_random_element(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than dataset"
    picks = []
    for n_ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks: # random inside
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [15]:
show_random_element(cut_datasets['train'])

,translation,subdataset
0,"{'en': 'oh, that's very nice of you.', 'th': 'โอ้ , คุณ น่ารัก มาก .'}",LST_Corpus
1,"{'en': None, 'th': 'อย่าง ​ ไร ​ ก็ ​ ตาม คู่มือ ​ การ ​ ดู ​ แล ​ ฟัน ​ สำหรับ ​ ครอบครัว ​ ที่ ​ ชื่อ ​ วิธี ​ รักษา ​ รอย ยิ้ม ​ ของ ​ ครอบครัว ​ คุณ ( ภาษา ​ อังกฤษ ) รายงาน ​ ว่า เรื่อง ​ ที่ ​ นับ ​ ว่า ​ สำคัญ ​ อย่าง ​ ยิ่ง ​ คือ ​ การ ​ คอย ​ ระวัง ​ ว่า ​ คุณ ​ กิน ​ ขนม ​ เมื่อ ​ ไร ​ และ ​ บ่อย ​ แค่ ​ ไหน .'}",LST_Corpus
2,"{'en': None, 'th': 'รายงาน ​ หนึ่ง ​ ใน ​ หนังสือ ​ พิมพ์ ​ เดอะ กา เซต ต์ แห่ง ​ เมือง ​ มอ นท รี ออล แคนาดา กล่าว ​ ว่า “ ใน ​ การ ​ ศึกษา ​ วิจัย ​ ที่ ​ ทำ ​ กับ ​ คน 4,600 คน ​ ที่ ​ มี ​ อายุ ​ ระหว่าง 18 - 30 ปี ” พวก ​ นัก ​ วิจัย “ พบ [ ว่า ] ยิ่ง ​ คน ​ ใด ​ มี ​ บุคลิกภาพ ​ ที่ ​ แสดง ​ ถึง ​ ความ ​ ไม่ ​ เป็น ​ มิตร , ความ ​ ข้อง ขัดใจ , และ ​ ความ ​ คิด ​ มุ่ง ​ ร้าย ​ มาก ​ เท่า ​ ใด ” ปอด ​ ของ ​ เขา ​ ก็ ​ จะ ​ ยิ่ง ​ อ่อนแอ ​ มาก ​ เท่า ​ นั้น .'}",LST_Corpus
3,"{'en': 'The result ? A weight equivalent to forty million elephants .', 'th': 'ผล ​ เป็น ​ อย่าง ​ ไร ?'}",LST_Corpus
4,"{'en': 'after rain comes fair weather.', 'th': 'หลัง ฝน อากาศ ดี ก็ เข้า มา .'}",LST_Corpus


In [16]:
cut_datasets['train'][0:2]

{'translation': [{'en': 'bangkok has so many fun attractions for tourists .',
   'th': 'กรุงเทพ มี สถานที่ ดึงดูด นักท่องเที่ยว มากมาย'},
  {'en': 'H .', 'th': 'H .'}],
 'subdataset': ['LST_Corpus', 'LST_Corpus']}

#### Metric

In [17]:
# metric

In [18]:
metric = load_metric("sacrebleu")
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## 1.2 Preprocessing the Data

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [20]:
# Set Source and Target Language

if 'mbart' in model_checkpoint:
    tokenizer.src_lang = 'th'
    tokenizer.tgt_lang = 'en'

In [21]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate Thai to English: "
elif model_checkpoint in ["Helsinki-NLP/opus-mt-en-mul"]:
    prefix = '>>tha<<'
else:
    prefix = ""

In [22]:
prefix

'>>tha<<'

In [23]:
max_input_len  = 128
max_target_len = 128

source_lang = 'en'
target_lang = 'th'

def preprocess_function(examples):
#     inputs = [prefix + ex[source_lang] for ex in examples['translation']]
    inputs, targets = [], []
    for ex_ in (examples['translation']):
        ex_target = ex_[target_lang]
        # SKIP NULL THING
        if ex_target is not None and ex_[source_lang] is not None:
            targets.append(ex_[target_lang])
            inputs.append(prefix + ex_[source_lang])
            
    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True) # Pad to longest word (128 char)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_len, truncation=True)
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [24]:
tokenized_datasets = cut_datasets.map(preprocess_function, 
                                      batched=True, 
                                      batch_size=1000, 
                                      num_proc=32, 
                                      remove_columns=["translation", "subdataset"], # Prevent ArrowInvalid Error (Skip)
                                     )

In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 849727
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 94520
    })
})

In [ ]:
tokenized_datasets['train'][100]

## 1.3 Fine-tuning the model

In [27]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/296M [00:00<?, ?B/s]

In [28]:
model_checkpoint

'Helsinki-NLP/opus-mt-en-mul'

In [29]:
torch.cuda.empty_cache()

In [30]:
!nvidia-smi

Tue Mar  8 12:52:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   39C    P0    34W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   41C    P0    36W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [33]:
model_name = model_checkpoint.split('/')[-1]
new_model_name = f'{model_name}-finetuned-1M-{source_lang}-to-{target_lang}'
print('New Model Name')
print(new_model_name)

New Model Name
opus-mt-en-mul-finetuned-1M-en-to-th


In [34]:
batch_size = 32
args = Seq2SeqTrainingArguments(
    new_model_name,
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
#     gradient_checkpointing=True, # For Accelerate Training
    weight_decay =0.01,
    save_total_limit = 3,
    num_train_epochs=1,
    predict_with_generate=True,
    dataloader_num_workers=32, # Multi-tread CPU
    fp16=True,
    eval_accumulation_steps = 10, # Reduce Using GPU Ram When Evaulation
    push_to_hub = False,
)

#### Create Data collator

In [35]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [36]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels= [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    print('preds', preds)
    print('labels', labels)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens = True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result['score']}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [37]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset= tokenized_datasets['train'],
#     eval_dataset = tokenized_datasets['test'],
    eval_dataset = None,
    data_collator = data_collator,
    tokenizer = tokenizer,
#     compute_metrics = compute_metrics,
)

Using amp half precision backend


## 1.4 *train time!*

In [ ]:
trainer.train()

In [39]:
trainer.save_model()

Saving model checkpoint to opus-mt-en-mul-finetuned-1M-en-to-th
Configuration saved in opus-mt-en-mul-finetuned-1M-en-to-th/config.json
Model weights saved in opus-mt-en-mul-finetuned-1M-en-to-th/pytorch_model.bin
tokenizer config file saved in opus-mt-en-mul-finetuned-1M-en-to-th/tokenizer_config.json
Special tokens file saved in opus-mt-en-mul-finetuned-1M-en-to-th/special_tokens_map.json


When Loading the model


``` 
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```



# 2. Using Translation Model

### Using AutoModel (With model selection)

In [40]:
?AutoModelForSeq2SeqLM.

Object `AutoModelForSeq2SeqLM.` not found.


In [41]:
# metric = load_metric("bleu")
# fake_preds = [["hello"],["there"], ["general"], ['kenobi']]
# fake_labels = [["hello"],["there"], ["general"], ['kenobi']]
# # fake_labels = [["hello there"], ["general kenobi"]]
# metric.compute(predictions=fake_preds, 
#                references=fake_labels)

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_pretrain = "Helsinki-NLP/opus-mt-en-mul"
new_model_name = 'TNANA-attacut-th-to-en-pt2/checkpoint-23000'
model_for_predict = f"./{new_model_name}"
# model_for_predict = "huak95/opus-mt-th-en-finetuned-5k-th-to-en"
print(model_for_predict)
model_pt = AutoModelForSeq2SeqLM.from_pretrained(model_pretrain)
model = AutoModelForSeq2SeqLM.from_pretrained(model_for_predict)
tokenizer = AutoTokenizer.from_pretrained(model_for_predict)

loading configuration file https://huggingface.co/Helsinki-NLP/opus-mt-en-mul/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1f25261d7e9a5b5e6ce297ed0729be5ee5cfcf28a0036e3f80eba1960bb4b2d9.d51a6de0226a95f56f1f97bb6b6aefc907cc894dd36c443cdd34f429f486314c
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-en-mul",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      64109
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 64109,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
 

./TNANA-attacut-th-to-en-pt2/checkpoint-23000


All model checkpoint weights were used when initializing MarianMTModel.

All the weights of MarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-mul.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MarianMTModel for predictions without further training.
loading configuration file ./TNANA-attacut-th-to-en-pt2/checkpoint-23000/config.json
Model config MarianConfig {
  "_name_or_path": "./TNANA-attacut-th-to-en-pt2/checkpoint-23000",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      62306
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62

In [11]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model,
    tokenizer = tokenizer,
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [12]:
!git lfs track 
model.push_to_hub('huak95/TNANA-attacut-th-to-en')

Configuration saved in huak95/TNANA-attacut-th-to-en/config.json
Model weights saved in huak95/TNANA-attacut-th-to-en/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 32.0k/290M [00:00<?, ?B/s]

To https://huggingface.co/huak95/TNANA-attacut-th-to-en
   d82f8ce..87859e5  main -> main



'https://huggingface.co/huak95/TNANA-attacut-th-to-en/commit/87859e56b8929f990770230f2a41da535388bbe3'

#### PUSH TO HUB >_<

In [50]:
!git lfs track 

# model.push_to_hub('huak95/opus-mt-en-mul-finetuned-1M-en-to-th')
tokenizer.push_to_hub('huak95/opus-mt-en-mul-finetuned-1M-en-to-th')

Listing tracked patterns
    opus-mt-th-en-finetuned-5k-th-to-en/* (.gitattributes)
Listing excluded patterns


tokenizer config file saved in huak95/opus-mt-en-mul-finetuned-1M-en-to-th/tokenizer_config.json
Special tokens file saved in huak95/opus-mt-en-mul-finetuned-1M-en-to-th/special_tokens_map.json
To https://huggingface.co/huak95/opus-mt-en-mul-finetuned-1M-en-to-th
   a317087..25fd024  main -> main



'https://huggingface.co/huak95/opus-mt-en-mul-finetuned-1M-en-to-th/commit/25fd0247496e203b908c6252a9e714cab6832fb5'

#### Testing >0<

In [45]:
def harry_translate(input_texts, model=model_pt):
    inputs = tokenizer('>>tha<<' + input_texts['en'], return_tensors = "pt")
    
    outputs = model.generate(inputs["input_ids"],
                             max_length=40,
                             num_beams=4,
                             early_stopping=True)

    return tokenizer.decode(outputs[0]).replace('<pad> ',''), input_texts['th']

In [48]:
datasets_size = len(cut_datasets['test']['translation'])
rand_idx = np.random.randint(datasets_size, size=10)

for i in rand_idx:
    print(f'index: {i}')

    text_ = cut_datasets['test']['translation'][i]
    thai_text = text_['en']
    pred_text_pt, real_text = harry_translate(text_, model_pt)
    pred_text, real_text = harry_translate(text_, model)
    
#     m_score = compute_metrics.compute(predictions=[pred_text], references=[real_text])

    print('thai_text ตัวอย่าง:\t',  thai_text)
    print('real_text เฉลย  :\t', real_text)
    print('pre_train ก่อนจูน :\t', pred_text_pt)
    print('pred_text หลังจูน :\t', pred_text)
    print()

index: 83104
thai_text ตัวอย่าง:	 grunt
real_text เฉลย  :	 เนื้อ สุกร
pre_train ก่อนจูน :	 กรอนท์
pred_text หลังจูน :	 กรุนต์

index: 53707
thai_text ตัวอย่าง:	 the party gonna be great
real_text เฉลย  :	 งาน ปาร์ตี้ จะ ต้อง เยี่ยม
pre_train ก่อนจูน :	 ปาร์ตี้จะดี
pred_text หลังจูน :	 งานเลี้ยง จะ เยี่ยม

index: 85305
thai_text ตัวอย่าง:	 This is no small problem , since making a hole in the wrong place can cause excessive bleeding .
real_text เฉลย  :	 นอก ​ จาก ​ นั้น นัก ​ เจาะ ​ หลาย ​ คน ​ ขาด ​ ความ ​ เข้าใจ ​ เรื่อง ​ กายวิภาคศาสตร์ .
pre_train ก่อนจูน :	 นี่ไม่ใช่ปัญหาเล็ก เนื่องจากการทําลายในสถานที่ผิด สามารถทําให้เลือด
pred_text หลังจูน :	 นี่ ไม่ ใช่ ปัญหา เล็ก เนื่อง จาก การ ทํา หลุม ใน ที่ ผิด อาจ ทํา

index: 28693
thai_text ตัวอย่าง:	 Likewise , for us to become spiritually sound , we need to “ chew ” on God’s utterances so that these can be assimilated into our mind and heart .
real_text เฉลย  :	 นอก ​ จาก ​ นี้ ​ ก็ ​ ต้อง ​ มี ​ การ ​ เคี้ยว ​ อาหาร ​ ให้ ​ ละเอียด ​ แล

#### VIEW NAN DATA

In [ ]:
text_df = pd.DataFrame(cut_datasets['test']['translation'])


In [ ]:
datasets_size = len(cut_datasets['test']['translation'])
rand_idx = np.random.randint(datasets_size, size=100)

datasets_small = text_df[text_df.en.isnull()]
for i in (datasets_small).index:
    print(f'index: {i}')


    thai_text = datasets_small['th'].loc[i]
    real_text = datasets_small['en'].loc[i]
#     pred_text_pt, real_text = harry_translate(text_, model_pt)
#     pred_text, real_text = harry_translate(text_, model)
    
#     m_score = compute_metrics.compute(predictions=[pred_text], references=[real_text])

    print('thai_text ตัวอย่าง:\t',  thai_text)
    print('real_text เฉลย  :\t', real_text)
#     print('pre_train ก่อนจูน :\t', pred_text_pt)
#     print('pred_text หลังจูน :\t', pred_text)
    print()

In [ ]:
datasets_small['len_th'] = datasets_small['th']

In [90]:
datasets_small['th']

21       พวก ​ เขา ​ หยั่ง ​ รู้ ​ เข้าใจ ​ ความ ​ เร่ง...
72       เป็น ​ ความ ​ เบิก บาน ​ ใจ ​ ใน ​ ทุก ​ วัน ​...
81       เก็บ ​ รายการ ​ ข้อ ​ คัมภีร์ ​ นี้ ​ ไว้ ​ ใน...
116      ตัว ​ อย่าง ​ เช่น พ่อ ​ แม่ ​ บาง ​ คน ​ รู้ส...
141      เธอ ​ จะ ​ ให้ ​ พวก ​ เรา ​ ช่วย ​ กัน ​ จ่าย...
                               ...                        
99903    หลัง ​ จาก ​ พระ ​ บุตร ​ สุด ​ ที่ ​ รัก ​ ขอ...
99920    “ ไม่ ​ ว่า ​ ฉัน ​ จะ ​ ลุก ​ หรือ ​ จะ ​ นอน...
99954                ( ราย ​ ละเอียด ​ ดู ​ จาก ​ วารสาร )
99955    ผู้ ​ ที่ ​ อ้าง ​ ว่า ​ เป็น ​ คริสเตียน ​ ได...
99978    เอียน — การ ​ เป็น ​ ลูก ​ ผู้ ​ ชาย ​ ไม่ ​ ไ...
Name: th, Length: 5300, dtype: object